<a href="https://colab.research.google.com/github/Ayanlola2002/Technidus_project/blob/master/ProjectWebscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Web Scraping

####  Propertypro is more flexible in terms of legal right as seen in the terms and conditions page.   

#### https://www.propertypro.ng/terms
#### https://nigeriapropertycentre.com/terms-of-use 

.

#### Import Beautiful Soup for scraping, 
####requests for making request to a website and 
#### re for regular expressions

In [ ]:
import requests, re
from bs4 import BeautifulSoup

#### Making a request to the website and extract its content (page source)

In [ ]:
r=requests.get("https://www.propertypro.ng/property-for-rent?search=gbagada")
c=r.content

#### Parsing page source using the Beautiful soup HTML parser
#### Find all property features on the page.
#### I did this by first expecting the web page

In [ ]:
#beautifulsoup object
soup=BeautifulSoup(c,"html.parser")

real=soup.find_all("div",{"class":"prop-features"})

In [ ]:
#checking features to remove
real[0].get_text().strip().split()

['4', 'bed', '4', 'bath', '5', 'toilet']

In [ ]:
re.findall("..bath",real[0].get_text().strip())#[0][0]

['4 bath']

#### changing div and class below to search for something else. 

In [ ]:
real=soup.find_all("div",{"class":"prop-features"})

In [ ]:
#https://www.propertypro.ng/property-for-rent?search=gbagada&page=1

In [ ]:
#trying to establish page number
items = int(re.findall("\d+",soup.find_all("div",{"class":"jumbotron m-hide"})[0].text.split("total of")[1][:6].replace(",","").strip())[0])
listings = 20
page_nr = int(items/listings)
page_nr

33

In [ ]:
#WebScrape Code
l=[]
location = ["gbagada","ikeja","surulere","ikeja","ogba","iyana ipaja","lekki","ajah","ikorodu"]


for place in location:
    base_url="https://www.propertypro.ng/property-for-rent?search="+place+ "&auto=&type=&bedroom=&max_price="
    r=requests.get(base_url+".html")
    c=r.content
    soup=BeautifulSoup(c,"html.parser")

    items = int(re.findall("\d+",soup.find_all("div",{"class":"jumbotron m-hide"})[0].text.split("total of")[1][:6].replace(",","").strip())[0])
    listings = 20 #This can be edited back to 20 items as stated on site. However your code will take a while to run
    page_nr = int(items/listings)
    
    #base_url="https://www.propertypro.ng/property-for-rent?search="+place+ "&auto=&type=&bedroom=&max_price="
    for page in range(1,int(page_nr),1):

        r=requests.get(base_url+".html"+"&page="+str(page))
        c=r.content

        soup=BeautifulSoup(c,"html.parser")
        
        classes = ["col-lg-6 col-md-6 col-sm-6 col-xs-12 prop-meta-data","col-lg-8 col-md-8 col-sm-7 col-xs-12 prop-meta-data text-left",
                   "col-lg-9 col-md-9 col-sm-12 col-xs-12 main-listing-cont"]
        for class_ in classes:
            real=soup.find_all("div",{"class":class_})

            for i in list(range(0,len(real))):
                d={}
                d['page']= page
                try:
                    d["location"] = real[i].find("h3",{"class":"pro-location"}).text.strip()
                except (IndexError,TypeError,AttributeError):
                    d["location"] = None
                try:
                    d["specific_location"] = real[i].find("h3",{"class":"pro-location"}).text.strip().split("gbagada")[0].replace("-","").strip()
                except(IndexError,TypeError,AttributeError):
                    d['specific_location'] = None
                try:
                    d["features"]=real[i].find("span",{"class":"prop-aminities float-left"}).text.strip()  
                except (AttributeError,IndexError) as e:
                    d["features"]= None
                try:
                    d["bedrooms"]= re.findall("..bed",real[i].find("span",{"class":"prop-aminities float-left"}).text.strip())[0][0]
                except (IndexError,TypeError,AttributeError) as e:
                    d["bedrooms"]= None 
                try:
                    d["bathrooms"]= re.findall("..bath",real[i].find("span",{"class":"prop-aminities float-left"}).text.strip())[0][0]
                except (IndexError,TypeError,AttributeError) as e:
                    d["bathrooms"]= None
                try:
                    d["toilets"]= re.findall("..toilet",real[i].find("span",{"class":"prop-aminities float-left"}).text.strip())[0][0]
                except (IndexError,TypeError,AttributeError) as e:
                    d["toilets"]=None
                try:
                    d["description"]=real[i].find("p",{"class":"pro-description"}).text.strip()
                except (IndexError,TypeError,AttributeError) as e:
                    d["description"]= None
                try:
                    d["other_description"]=real[i].find("p",{"class":"pro-description readmore"}).text.strip()
                except (IndexError,TypeError,AttributeError) as e:
                    d["other_description"]= None     
                
                try:
                    d["price"]=real[i].find("p",{"class":"prop-price"}).text.strip().replace("₦","").replace(",","")
                except (IndexError,TypeError,AttributeError) as e:
                    d["price"] = None
                l.append(d)
                #print(l)
                #print(" "

#### Convert output to dataframe

In [ ]:
import pandas as pd
ld = pd.DataFrame(l)
ld

,page,location,specific_location,features,bedrooms,bathrooms,toilets,description,other_description,price
0,1,ifako gbagada gbagada lagos,ifako,2 bed 2 bath 2 toilet,2,2,2,"Decent 2 bedrooms ground flat in a block of 4,...","Decent 2 bedrooms ground flat in a block of 4,...",800000
1,1,ifako ifako gbagada gbagada lagos,ifako ifako,2 bed 2 bath 3 toilet,2,2,3,"2 bedroom apartment in a block of 4, upstairs ...","2 bedroom apartment in a block of 4, upstairs ...",1000000
2,1,millenuim ups gbagada lagos,millenuim ups,3 bed 3 bath 4 toilet,3,3,4,two tenants in the compound ... Security. Park...,two tenants in the compound ... Security. Park...,1800000
3,1,pedro-gbagada area gbagada lagos,pedro,2 bed 2 bath 3 toilet,2,2,3,Serviced 2 Bedroom all rooms en-suite flat wit...,Serviced 2 Bedroom all rooms en-suite flat wit...,1300000
4,1,back of mobil filling station by domino soluyi...,back of mobil filling station by domino soluyi,3 bed 3 bath 3 toilet,3,3,3,"Very Decent 3brm Flat with 3t, 3b, 2 rooms are...","Very Decent 3brm Flat with 3t, 3b, 2 rooms are...",900000
...,...,...,...,...,...,...,...,...,...,...
5146,11,None,None,None,None,None,None,None,None,None
5147,12,None,None,None,None,None,None,None,None,None
5148,13,None,None,None,None,None,None,None,None,None
5149,14,None,None,None,None,None,None,None,None,None


In [ ]:
scrapedata=ld.to_csv("data1.csv",index=False)

In [ ]:
#data check
ld.head(10)

,page,location,specific_location,features,bedrooms,bathrooms,toilets,description,other_description,price
0,1,ifako gbagada gbagada lagos,ifako,2 bed 2 bath 2 toilet,2,2,2,"Decent 2 bedrooms ground flat in a block of 4,...","Decent 2 bedrooms ground flat in a block of 4,...",800000
1,1,ifako ifako gbagada gbagada lagos,ifako ifako,2 bed 2 bath 3 toilet,2,2,3,"2 bedroom apartment in a block of 4, upstairs ...","2 bedroom apartment in a block of 4, upstairs ...",1000000
2,1,millenuim ups gbagada lagos,millenuim ups,3 bed 3 bath 4 toilet,3,3,4,two tenants in the compound ... Security. Park...,two tenants in the compound ... Security. Park...,1800000
3,1,pedro-gbagada area gbagada lagos,pedro,2 bed 2 bath 3 toilet,2,2,3,Serviced 2 Bedroom all rooms en-suite flat wit...,Serviced 2 Bedroom all rooms en-suite flat wit...,1300000
4,1,back of mobil filling station by domino soluyi...,back of mobil filling station by domino soluyi,3 bed 3 bath 3 toilet,3,3,3,"Very Decent 3brm Flat with 3t, 3b, 2 rooms are...","Very Decent 3brm Flat with 3t, 3b, 2 rooms are...",900000
5,1,shadia estate behind chevron staff club soluyi...,shadia estate behind chevron staff club soluyi,2 bed 2 bath 2 toilet,2,2,2,"Very nice 2brm flat bungalow with 2t, 2b, 1roo...","Very nice 2brm flat bungalow with 2t, 2b, 1roo...",1000000
6,1,iyana oworo oworonshoki gbagada lagos,iyana oworo oworonshoki,bed 1 bath 1 toilet,None,1,1,"Newly built room self contain with kitchen,toi...","Newly built room self contain with kitchen,toi...",300000
7,1,after deperlife church gbagada lagos,after deperlife church,1 bed 1 bath 1 toilet,1,1,1,Mini flat (1bedroom Apartment with kitchen toi...,Mini flat (1bedroom Apartment with kitchen toi...,450000
8,1,odo-eran iyana orowo gbagada oworonshoki gbaga...,odoeran iyana orowo,2 bed 2 bath 3 toilet,2,2,3,Newly built two bedroom flat with all rooms en...,Newly built two bedroom flat with all rooms en...,550000
9,1,"community street ,odo eran,iyana oworo gbagada...","community street ,odo eran,iyana oworo",1 bed 1 bath 1 toilet,1,1,1,"For lease: Newly built mini flat with kitchen,...","For lease: Newly built mini flat with kitchen,...",400000


In [ ]:
#check rows with null values
sum(ld.apply(lambda x: sum(x.isnull().values), axis = 0)>0)

9

In [ ]:
#removing all null values
ld = ld.dropna(how='any',axis=0) 

In [ ]:
#checking for null values
ld.isnull().sum()

page                 0
location             0
specific_location    0
features             0
bedrooms             0
bathrooms            0
toilets              0
description          0
other_description    0
price                0
dtype: int64

In [ ]:
#ld['location'] = ld['location'].str.extract(r'(gbagada|ikeja|surulere|ogba|iyana ipaja|lekki|ajah|ikorodu)').map({'gbagada':'gbagada','ikeja':'ikeja','ogba':'ogba','iyana ipaja':'iyana ipaja','lekki':'lekki','ajah':'ajah','ikorodu':'ikorodu'})

In [ ]:
#changing location content
ld["location"][ld['location'].str.contains("gbagada")] ="gbagada"
ld["location"][ld['location'].str.contains("ikeja")] ="ikeja"
ld["location"][ld['location'].str.contains("iyana ipaja")] ="iyana ipaja"
ld["location"][ld['location'].str.contains("surulere")] ="surulere"
ld["location"][ld['location'].str.contains("ogba")] ="ogba"
ld["location"][ld['location'].str.contains("lekki")] ="lekki"
ld["location"][ld['location'].str.contains("ajah")] ="ajah"
ld["location"][ld['location'].str.contains("ikorodu")] ="ikorodu"

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/

In [ ]:
#testing the values
ld['location'][ld['location']=='ogba']

1724    ogba
1725    ogba
1726    ogba
1727    ogba
1728    ogba
        ... 
1905    ogba
1908    ogba
1909    ogba
5102    ogba
5111    ogba
Name: location, Length: 161, dtype: object

In [ ]:
ld.head()

,page,location,specific_location,features,bedrooms,bathrooms,toilets,description,other_description,price
0,1,gbagada,ifako,2 bed 2 bath 2 toilet,2,2,2,"Decent 2 bedrooms ground flat in a block of 4,...","Decent 2 bedrooms ground flat in a block of 4,...",800000
1,1,gbagada,ifako ifako,2 bed 2 bath 3 toilet,2,2,3,"2 bedroom apartment in a block of 4, upstairs ...","2 bedroom apartment in a block of 4, upstairs ...",1000000
2,1,gbagada,millenuim ups,3 bed 3 bath 4 toilet,3,3,4,two tenants in the compound ... Security. Park...,two tenants in the compound ... Security. Park...,1800000
3,1,gbagada,pedro,2 bed 2 bath 3 toilet,2,2,3,Serviced 2 Bedroom all rooms en-suite flat wit...,Serviced 2 Bedroom all rooms en-suite flat wit...,1300000
4,1,gbagada,back of mobil filling station by domino soluyi,3 bed 3 bath 3 toilet,3,3,3,"Very Decent 3brm Flat with 3t, 3b, 2 rooms are...","Very Decent 3brm Flat with 3t, 3b, 2 rooms are...",900000
